# Voiceshell Work Notebook

*Adapted from Allison Parrish's Understanding Word Vectors notebook*

#### Python Package Installation for Speech Recognition

*Run in terminal to install SpeechRecognition.py & PocketSphinx engine:*

`pip install SpeechRecognition`

`pip install pocketsphinx`

#### Importing required libraries:

In [15]:
# IMPORTS
import math
import random
from __future__ import unicode_literals
import spacy
import speech_recognition as sr
import os
import numpy as np
from numpy import dot
from numpy.linalg import norm

#### Defining basic mathematical functions for vector math:

In [2]:
def meanv(coords):
    sumv = [0] * len(coords[0])
    for item in coords:
        for i in range(len(item)):
            sumv[i] += item[i]
    mean = [0] * len(sumv)
    for i in range(len(sumv)):
        mean[i] = float(sumv[i]) / len(coords)
    return mean

In [3]:
def cosine(v1, v2):
    if norm(v1) > 0 and norm(v2) > 0:
        return dot(v1, v2) / (norm(v1) * norm(v2))
    else:
        return 0.0

#### Defining Spacy/Word Vector low level functions:

In [4]:
def sentvec(s):
    sent = nlp(s)
    return meanv([w.vector for w in sent])

In [5]:
def spacy_closest_sent(space, input_str, n=10):
    input_vec = sentvec(input_str)
    return sorted(space,
                  key=lambda x: cosine(np.mean([w.vector for w in x], axis=0), input_vec),
                  reverse=True)[:n]

#### Creating Spacy processor and loading a corpus:

In [6]:
nlp = spacy.load('en')
doc = nlp(open("revisedCorpus/allPoems.txt").read())

#### Splitting corpus into individual, complete sentences:

In [7]:
sentences = list(doc.sents)

#### Define list of prompts to user:

In [8]:
inquiries = ["How are you feeling?", "Tell me a quote you like.", "What are the first three words that pop into your head?", "What day is it today?", "What's your favourite colour?", "Where are you right now?", "What's your job?", "What do you like to eat?", "What animal would you want as a pet?"]

#### Defining user input prompts, and getting input to process, return result:

In [9]:
print(inquiry)
user_input = input()

for sent in spacy_closest_sent(sentences, user_input):
    print("---")
    print(sent.text.replace('\n',' '))
    print("---")

NameError: name 'inquiry' is not defined

#### Setting up basic SpeechRecognition input:
[Source](http://www.codesofinterest.com/2017/03/python-speech-recognition-pocketsphinx.html)

In [21]:
# get microphone input
r = sr.Recognizer()
with sr.Microphone() as source:
    print("Just a moment, calibrating the microphone...")
    # 5 second listen and find ambient noise level
    r.adjust_for_ambient_noise(source, duration=5)
    inquiry = random.choice(inquiries)
    
    # read question aloud and print it
    os.system("say '" + inquiry + "'")
    print(inquiry," (Speak aloud, please.)")
    audio = r.listen(source)
    
# recognise speech with Sphinx
try:
    recogSpeech = r.recognize_sphinx(audio) + "'"
    print("I heard you say: ", recogSpeech)
    for sent in spacy_closest_sent(sentences, recogSpeech):
        print("----------")
        print("...")
        print(sent.text)
        print("----------")
except sr.UnknownValueError:
    print("Sorry, I couldn't understand you.")
except sr.RequestError as e:
    print("Sphinx error; {0}".format(e))

Just a moment, calibrating the microphone...
I heard you say:  blink run fast'
----------
...
you say with your Pak 'n Save poi. 

----------
----------
...
It takes time
to know whether to pause
or hurry, head down.


----------
----------
...
So you run 
and halfway you sink 
to your waist 
and down the dirt road 
come The Poi Girls, slowing 
to a stop. 

----------
----------
...
I will not race you 
to the finish line 
race you to freedom 
I will not count down.


----------
----------
...
I let it go floating 
it bobbed and the sun caught the eye, closing. 

----------
----------
...
*

You know how it is

to give up the piss 
a week to the

day before Christmas

you know how it is

to fall over sober 
safe in some spot,

come to later 
remembering the lot.

*

the rugby ball kicked 
far as the far paddock

where an apple tree caught it. 

----------
----------
...
If and when I get lost, you can slide me into the red box.


----------
----------
...
A brown
horizon turning
fast i

In [20]:
os.system("say '" + inquiry + "'")

0